# Learning Notebook - Part 3 of 3 - Dealing with larger datasets

## 1. Introduction

Up to this point, you've already handled several datasets. Most were small. When a dataset is bigger Pandas could have problems diggesting all of it. 

![title](./media/panda_eating.jpg)

Here we're going to handle larger datasets using pandas. Let's start by doing a couple of read tricks

* **Only reading n lines**

Sometimes we're dealing with large files and we're just interested in previewing our data.
With argument **nrows**, we can specify how many lines of the files should be read into the DataFrame. The n first lines of the file will be read.

In [6]:
import os
import pandas as pd
import random

In [7]:
# Again an helper function to get filepaths
def pokemons_filepath(filename):
    return os.path.join('data', 'pokemons', filename)

In [8]:
# Number of lines in the pokemons file
pd.read_csv(pokemons_filepath('pokemons.csv')).shape[0]

800

In [9]:
pd.read_csv(pokemons_filepath('pokemons.csv'), nrows=3)

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False


* **Only reading some columns**

If we only want to read some columns, we can use argument **usecols** giving it a list of column names or numbers.

In [10]:
pd.read_csv(pokemons_filepath('pokemons_short.csv'), usecols=[1, 3, 7])

,Name,Type 2,Sp. Atk
0,Bulbasaur,Poison,65
1,Ivysaur,Poison,80
2,Venusaur,Poison,100
3,Mega Venusaur,Poison,122
4,Charmander,NaN,60


* **Read n random lines**

To read a set of *n* random lines from a file, we can use the little trick using the skiprows argument of teh read_csv function! Let's see how.

First, we need to know how many lines does the file have. Let's do it with a shell command, as we saw in the first part of this BLU.

In [11]:
lines_in_file = ! wc -l < data/pokemons/pokemons.csv
lines_in_file = int(lines_in_file[0])-1
lines_in_file

800

Now let's imagine we want a sample of only 10 random rows from the file. We're going to use `skiprows` to... well... skip a couple of rows. The trick is to randomly pick a set of rows to be skipped, while leaving 10 rows in place.

In [12]:
sample_number = 10
n_rows_to_skip = lines_in_file - sample_number

 # this is to get always the same sample. can be removed if we want the sample to change
rows_to_skip = random.sample(
    range(1, lines_in_file-1), # this is a range from the first row after the header, to the last row on the file
    n_rows_to_skip # this is the number of rows we want to sample, i.e, to skip
)

pd.read_csv( 
    pokemons_filepath('pokemons.csv'),
    skiprows=rows_to_skip
)

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,145,Eevee,Normal,NaN,55,55,50,45,65,55,1,False
1,283,Swampert,Water,Ground,100,110,90,85,90,60,3,False
2,364,Cacturne,Grass,Dark,70,115,60,115,60,55,3,False
3,441,Empoleon,Water,Steel,84,86,88,111,101,60,4,False
4,458,Burmy,Bug,NaN,40,29,45,29,45,36,4,False
5,471,Gastrodon,Water,Ground,111,83,68,92,82,39,4,False
6,482,Chingling,Psychic,NaN,45,30,50,65,50,45,4,False
7,525,Mamoswine,Ice,Ground,110,130,80,70,60,80,4,False
8,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True
9,800,Volcanion,Fire,Water,80,110,120,130,90,70,6,True


The downside of the approach we presented above, is that you have to open the file twice (one for getting the number of rows and another to take the sample).

If you don't care a lot about the sample size, and you're happy taking a sample that is certain percentage of the rows in the file, you can actually avoid opening the file twice.

In this case, we will give a function to the `skiprows` argument of read_csv. This function will be called for each row in the file, and each row will be skipped with a probablility of `1 - p`. The `i > 0`is to avoid skipping the header.

In [13]:
# sample 1% of the rows
p = 0.01
random.seed(42)

pd.read_csv( 
    pokemons_filepath('pokemons.csv'), 
    skiprows=lambda i: i > 0 and random.random() > p
)

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,20,Mega Beedrill,Bug,Poison,65,150,40,15,80,145,1,False
1,125,Mega Kangaskhan,Normal,NaN,105,125,100,60,100,100,1,False
2,270,Lugia,Psychic,Flying,106,90,130,90,154,110,2,True
3,291,Beautifly,Bug,Flying,60,70,50,100,50,65,3,False
4,298,Nuzleaf,Grass,Dark,70,70,40,60,40,60,3,False
5,369,Seviper,Poison,NaN,73,100,60,100,60,65,3,False
6,395,Wynaut,Psychic,NaN,95,23,48,23,48,23,3,False
7,428,Jirachi,Steel,Psychic,100,100,100,100,100,100,3,True
8,482,Chingling,Psychic,NaN,45,30,50,65,50,45,4,False
9,685,Golurk,Ground,Ghost,89,124,80,55,80,55,5,False


## 2. Load the file using chunks

Sometimes files are very large, but you still want to go through all the precious data stored there.
The problem is that you can easily fill up your memory and processing the data in the file becomes really slow!

In these cases you can use chunks! Using chunks means loading the data in small portions at each time, and applying transformations to those small portions of data. This shold allow you to process the data in large files faster.

This is a simple toy example:

In [15]:
def filter_type1_ice(data):
    return data.loc[data['Type 1'] =='Ice',:]

In [14]:
chunks_iter = pd.read_csv(
    pokemons_filepath('pokemons.csv'),
    chunksize=99
)

chunk_arr = []
for data_chunk in chunks_iter:
    chunk_arr.append(data_chunk)
    
print(
    "We analyized a total of",
    sum([len(c) for c in chunk_arr]), 
    "rows divided in", 
    len(chunk_arr), 
    " chunks with the following configuration:\n",
    [len(c) for c in chunk_arr]
)

We analyized a total of 800 rows divided in 9  chunks with the following configuration:
 [99, 99, 99, 99, 99, 99, 99, 99, 8]


Since `pd.read_csv` with chunksize returns a generator, you can handle each chunk with small amount of memory. Know more about generators and memory advantages [here](https://wiki.python.org/moin/Generators). 

Let's now see an example with real data.
We'll load our pokemons file in chunks and, in each chunk we'll only keep the rows where the pokemon has `Type 1 = Ice`. 
Then, we'll create a DataFrame with the filtered chunks.

In [17]:
chunks_iter = pd.read_csv(
    pokemons_filepath('pokemons.csv'),
    chunksize=99
)

chunk_arr = []
for data_chunk in chunks_iter:
    data_chunk_filtered = filter_type1_ice(data_chunk)
    chunk_arr.append(data_chunk_filtered)

final_pd = pd.concat(chunk_arr, axis=0)
final_pd.head()
final_pd 

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
133,134,Jynx,Ice,Psychic,65,50,35,115,95,95,1,False
156,157,Articuno,Ice,Flying,90,85,100,95,125,85,1,True
238,239,Swinub,Ice,Ground,50,50,40,30,30,50,2,False
239,240,Piloswine,Ice,Ground,100,100,80,60,60,50,2,False
243,244,Delibird,Ice,Flying,45,55,45,65,45,75,2,False
257,258,Smoochum,Ice,Psychic,45,30,15,85,65,65,2,False
395,396,Snorunt,Ice,NaN,50,50,50,50,50,50,3,False
396,397,Glalie,Ice,NaN,80,80,80,80,80,80,3,False
397,398,Mega Glalie,Ice,NaN,80,120,80,120,80,100,3,False
398,399,Spheal,Ice,Water,70,40,50,55,50,25,3,False



**Note**

Of course we are applying big files techniques to small files! Given this file size we could simply apply 
```
data = pd.read_csv(pokemons_filepath('pokemons.csv'))
data.loc[data['Type 1'] =='Ice',:]

```
**End of Note**

---


If, for some reason you still need to reduce the resulting DataFrame size, there are still 2 things you can do: 
- reduce the number of columns
- change field types to the most appropriate ones, eg, you can "shrink" ints and floats to 32 bits or even 16

Here is an example of this and how we can measure the memory "savings".

In [12]:
# size in bytes
final_pd.memory_usage().sum()

2328

Here we remove some columns, and the DataFrame size was already reduced.

In [13]:
cols = ['Name', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation']
final_pd_shrinked = final_pd.loc[:,cols]
final_pd_shrinked.memory_usage().sum()

1536

Here we change some column types, from `int64` to `int16`:

In [14]:
cols = ['Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation']
final_pd_shrinked[cols] = final_pd_shrinked[cols].astype('int16')

#size in bytes
final_pd_shrinked.memory_usage().sum()

672